In [1]:
# Importing libraries
import gymnasium as gym
import numpy as np
from itertools import count
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [3]:
env = gym.make('CartPole-v1')

In [4]:
print("There are {} actions".format(env.action_space.n))

There are 2 actions


In [5]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, 128) # 4 parameters in the observation space
        self.actor = nn.Linear(128, 2) # 2 possible actions
        self.critic = nn.Linear(128, 1) # Single action value

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_prob = F.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_prob, state_values

In [6]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()
# Decide whether to move block left or right based on model output probabilities

In [7]:
def finish_episode():
    # Calculating losses and performing backprop
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    returns = []

    for r in model.rewards[::-1]:
        R = r + 0.99 * R # Gamma is 0.99
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    loss.backward()
    optimizer.step()

    del model.rewards[:]
    del model.saved_actions[:]

In [8]:
model = ActorCritic()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [9]:
def train():
    running_reward = 10
    for i_episode in count():
        state, info = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, terminated, truncated, info = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if terminated or truncated:
                break

        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0:
            print(f"Episode {i_episode} Reward: {ep_reward:.2f} Average reward: {running_reward:.2f}")

        if running_reward > env.spec.reward_threshold:
            print(f"Solved, running reward is now {running_reward} and the last episode runs to {t} timesteps")
            break

In [10]:
train()

Episode 0 Reward: 12.00 Average reward: 10.10
Episode 10 Reward: 29.00 Average reward: 21.14
Episode 20 Reward: 66.00 Average reward: 37.12
Episode 30 Reward: 16.00 Average reward: 43.68
Episode 40 Reward: 16.00 Average reward: 32.50
Episode 50 Reward: 33.00 Average reward: 29.84
Episode 60 Reward: 63.00 Average reward: 36.02
Episode 70 Reward: 95.00 Average reward: 77.25
Episode 80 Reward: 34.00 Average reward: 60.08
Episode 90 Reward: 44.00 Average reward: 67.65
Episode 100 Reward: 26.00 Average reward: 55.39
Episode 110 Reward: 28.00 Average reward: 44.77
Episode 120 Reward: 288.00 Average reward: 52.37
Episode 130 Reward: 146.00 Average reward: 64.60
Episode 140 Reward: 189.00 Average reward: 96.45
Episode 150 Reward: 176.00 Average reward: 106.07
Episode 160 Reward: 180.00 Average reward: 135.01
Episode 170 Reward: 81.00 Average reward: 143.93
Episode 180 Reward: 270.00 Average reward: 143.04
Episode 190 Reward: 89.00 Average reward: 144.53
Episode 200 Reward: 63.00 Average reward

KeyboardInterrupt: 